---

# Assignment 5 - GitHub and the ticketmaster.com API

##### Marius Knipp (Student ID: 6290375)
##### January 25, 2023

In [1]:
import numpy as np
import pandas as pd
import requests

---
### Getting to know the API

6) Whithin the scope of this assignment, you do not have to request your own API key. Instead retrieve a valid key from the API Explorer. This API key enables you to perform the GET requests needed throughout this assignment. Even though this API key is not secret per se (it is publicly visible on the API Explorer website), please comply to the common secrecy practices discussed in the lecture and the tutorial: Treat the API key as a secret token. Your API key should neither appear in the code that you are submitting nor in your public GitHub repository.

In [2]:
with open('ticketmaster_key.py') as script:
    exec(script.readline())

---

### Interacting with the API - the basics

7) Perform a first `GET` request, that searches for event venues in Germany (`countryCode = "DE"`). Extract the content from the response object and inspect the resulting list. Describe what you can see.

In [3]:
content = requests.get('https://app.ticketmaster.com/discovery/v2/venues',
                       params={'apikey': ticketmaster_key, 
                               'locale': '*',
                               'countryCode': 'DE'}).json()

In [4]:
content['page']

{'size': 20, 'totalElements': 12932, 'totalPages': 647, 'number': 0}

In [5]:
content['_embedded']['venues'][0].keys()

dict_keys(['name', 'type', 'id', 'test', 'url', 'locale', 'images', 'postalCode', 'timezone', 'city', 'country', 'address', 'location', 'markets', 'dmas', 'boxOfficeInfo', 'upcomingEvents', 'ada', '_links'])

*Answer: The list contains 20 venues in Germany with various information about each venue (e.g. name, URL, city etc.). These 20 venues are only on the first page. In total, there are 647 pages with 20 venues each, adding up to 12932 venues in Germany.*

8) Extract the `name`, the `city`, the `postalCode` and `address`, as well as the `url` and the `longitude` and `latitude` of the venues to a data frame.

In [6]:
venues_list = []
venues = content['_embedded']['venues']

for venue in venues:
    venue_dict = {}
    venue_dict['name'] = venue.get('name', np.nan)
    venue_dict['city'] = venue['city']['name'] if venue.get('city', None) else np.nan
    venue_dict['postalCode'] = venue.get('postalCode', np.nan)
    venue_dict['address'] = venue['address']['line1'] if venue.get('address', None) else np.nan
    venue_dict['url'] = venue.get('url', np.nan)
    venue_dict['longitude'] = venue['location']['longitude'] if venue.get('location', None) else np.nan
    venue_dict['latitude'] = venue['location']['latitude'] if venue.get('location', None) else np.nan
    venues_list.append(venue_dict)


df = pd.DataFrame(venues_list)
df.head()

,name,city,postalCode,address,url,longitude,latitude
0,Grosse Freiheit 36,Hamburg,22767,Grosse Freiheit 36,http://www.ticketmaster.de/venue/287636,9.957905,53.551187
1,Gruenspan,Hamburg,22767,Grosse Freiheit 58,http://www.ticketmaster.de/venue/287155,9.958075,53.551885
2,Huxleys Neue Welt,Berlin,10967,Hasenheide 107 – 113,http://www.ticketmaster.de/venue/286842,13.42138,52.486391
3,Zentrum am Park,Emmelshausen,56281,Rhein-Mosel-Straße 45,https://www.ticketmaster.de/venue/zentrum-am-p...,7.55656,50.15544
4,Stadthalle (Theatersaal),Mülheim,45479,Theodor-Heuss-Platz 1,https://www.ticketmaster.de/venue/stadthalle-t...,6.87471,51.42778
